# Ý tưởng cho tiền xử lý dữ liệu
1. Đưa toàn bộ type về dạng số
2. Xử lý dữ liệu thiếu bằng cách điền giá trị thay thế
3. Scale dữ liệu

#### Các thư viện sử dụng
- pandas: làm việc với file csv
- sklearn.preprecessing: để tận dụng cài đặt encoding của sklearn

In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

Đọc file csv

In [54]:
data = pd.read_csv("../data/csv/train.csv")
data

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


# Đưa dữ liệu về dạng số
Cần xem xét sử dụng One hot encoding hay Label encoding.
- One hot encoding: dùng khi giá trị phân loại không có quan hệ thứ tự như seasons
- Label encoding: dùng khi giá trị phân loại có thứ tự như Low - Med - High

Tìm ra các cột mà dữ liệu ở đấy là string, quan sát thấy đều là các cột seasons

In [55]:
string_columns = data.select_dtypes(include=['object'])
string_column_labels = string_columns.columns[1:].tolist() # Bỏ qua field id
string_column_labels

['Basic_Demos-Enroll_Season',
 'CGAS-Season',
 'Physical-Season',
 'Fitness_Endurance-Season',
 'FGC-Season',
 'BIA-Season',
 'PAQ_A-Season',
 'PAQ_C-Season',
 'PCIAT-Season',
 'SDS-Season',
 'PreInt_EduHx-Season']

In [56]:
# Bước này làm cho vui :)) tại file test.csv thiếu cột nên nhiều cái phải thủ công
season_columns = ['Basic_Demos-Enroll_Season',
 'CGAS-Season',
 'Physical-Season',
 'Fitness_Endurance-Season',
 'FGC-Season',
 'BIA-Season',
 'PAQ_A-Season',
 'PAQ_C-Season',
 'PCIAT-Season',
 'SDS-Season',
 'PreInt_EduHx-Season']

- Cột seasons -> không có quan hệ thứ tự -> dùng One-hot encoding
- Ngoài ra, để đảm bảo không xảy ra trường hợp tồn tại cột nào đấy mà toàn bộ rows không thiếu mùa (tức là không đủ 4 mùa Xuân Hạ Thu Đông thì lúc encode sẽ không phải là vector 4D [x, y, z, t]) -> để nhất quán và tránh lỗi sau này thì mạnh dạn các trường để encode là 4 mùa luôn

In [57]:
all_seasons = ['Spring', 'Summer', 'Fall', 'Winter']

encoded_data = data.copy()
# Duyệt qua các cột mùa và thực hiện One-Hot Encoding
for column in season_columns:
    # Áp dụng pd.get_dummies để One-Hot Encoding
    one_hot = pd.get_dummies(encoded_data[column], prefix=column)
    
    # Đảm bảo các mùa đều có mặt trong One-Hot Encoding (bổ sung cột thiếu)
    one_hot = one_hot.reindex(columns=[f"{column}_{season}" for season in all_seasons], fill_value=0)
    
    # Thêm các cột One-Hot vào bản sao dữ liệu và xóa cột ban đầu trong bản sao
    encoded_data = pd.concat([encoded_data, one_hot], axis=1)
    encoded_data = encoded_data.drop(columns=[column])

encoded_data


,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,PCIAT-Season_Fall,PCIAT-Season_Winter,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Fall,SDS-Season_Winter,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Winter
0,00008ff9,5,0,51.0,16.877316,46.0,50.8,NaN,NaN,NaN,...,True,False,False,False,False,False,False,False,True,False
1,000fd460,9,0,NaN,14.035590,48.0,46.0,22.0,75.0,70.0,...,True,False,False,False,True,False,False,True,False,False
2,00105258,10,1,71.0,16.648696,56.5,75.6,NaN,65.0,94.0,...,True,False,False,False,True,False,False,True,False,False
3,00115b9f,9,0,71.0,18.292347,56.0,81.6,NaN,60.0,97.0,...,False,False,False,True,False,False,False,False,False,True
4,0016bb22,18,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,13,0,60.0,16.362460,59.5,82.4,NaN,71.0,70.0,...,False,True,False,False,False,True,False,False,True,False
3956,ffa9794a,10,0,NaN,18.764678,53.5,76.4,27.0,60.0,78.0,...,False,False,False,False,False,False,False,False,False,True
3957,ffcd4dbd,11,0,68.0,21.441500,60.0,109.8,NaN,79.0,99.0,...,False,True,False,False,False,True,False,False,True,False
3958,ffed1dd5,13,0,70.0,12.235895,70.7,87.0,NaN,59.0,61.0,...,False,False,True,False,False,False,True,False,False,False


# Xử lý dữ liệu thiếu

Tìm các cột thiếu dữ liệu